In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys
import os

# Instead of importing, copy the SimpleOHLCGenerator class 
# that you shared earlier into your current file
# Or use this simpler import:
sys.path.append("/Users/newuser/Projects/robust_algo_trader/drl/")
from ohlc_generator import SimpleOHLCGenerator  # Remove 'drl.' prefix

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv("/Users/newuser/Projects/robust_algo_trader/data/gen_alpaca_data/AAPL_D1_raw_data.csv")
    generator = SimpleOHLCGenerator(df)
    synthetic_data = generator.generate_bootstrap_data(num_samples=10, segment_length=5)[1]
    fig = generator.plot_comparison(synthetic_data, num_bars=130)
    plt.show()

In [ ]:
synthetic_data